In [1]:
import os
os.chdir("../")
%pwd

'd:\\python-projects\\chest-cancer-classification'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class LogConfig:
    root_dir: Path
    tensorboard: Path
    lightning: Path
    mlflow: Path


@dataclass
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [3]:
from src.cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])


    
    def get_log_config(self) -> LogConfig:
        logs = self.config.logs
        create_directories([Path(logs.root_dir)])

        logs_config = LogConfig(
            root_dir=Path(logs.root_dir),
            tensorboard=Path(logs.tensorboard),
            lightning=Path(logs.lightning),
            mlflow = Path(logs.mlflow)
        )

        return logs_config



    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = Path(self.config.data_ingestion.unzip_dir) / "Chest-CT-Scan-data"
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE
        )

        return training_config

In [5]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from src.cnnClassifier.components.prepare_base_model import MyImageClassifier
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger, MLFlowLogger
from pytorch_lightning import Trainer

In [6]:
class TrainingDataModule(pl.LightningDataModule):
    def __init__(self, config: TrainingConfig):
        super().__init__()
        self.config = config


    def setup(self, stage=None):
        transform_list = [
            transforms.Resize(self.config.params_image_size[:-1]),
            transforms.ToTensor()
        ]
        if self.config.params_is_augmentation:
            transform_list = [
                transforms.RandomHorizontalFlip(),
                transforms.RandomRotation(20),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            ] + transform_list

        transform = transforms.Compose(transform_list)
        dataset = datasets.ImageFolder(self.config.training_data, transform=transform)

        val_size = int(0.2 * len(dataset))
        train_size = len(dataset) - val_size

        generator = torch.Generator().manual_seed(42)
        self.train_dataset, self.val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.config.params_batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.config.params_batch_size)

In [7]:
class Training:
    def __init__(self, config: TrainingConfig, logs: LogConfig):
        self.config = config
        self.logs = logs
        self.model = None
        self.datamodule = None

    def get_base_model(self):
        base_model = torch.load(self.config.updated_base_model_path, weights_only=False)
        self.model = MyImageClassifier(model=base_model, config=self.config)

    def get_data_module(self):
        self.datamodule = TrainingDataModule(self.config)

    def train(self):
        self.get_base_model()
        self.get_data_module()

        tb_logger = TensorBoardLogger(self.logs.tensorboard)
        mlflow_logger = MLFlowLogger(
            experiment_name="ChestCancerClassification",
            tracking_uri= str(self.logs.mlflow)
            )
        
        trainer = Trainer(
            max_epochs=self.config.params_epochs,
            accelerator="auto",
            logger=[tb_logger, mlflow_logger],
            enable_progress_bar=True,
            log_every_n_steps=1
        )

        trainer.fit(self.model, datamodule=self.datamodule)

        torch.save(self.model.model, self.config.trained_model_path)

In [8]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    logs_config = config.get_log_config()
    training = Training(config=training_config, logs= logs_config)
    training.train()
except Exception as e:
    raise e

[2025-06-09 11:44:35,536] [13] [common] - INFO - YAML file loaded successfully: config\config.yaml
[2025-06-09 11:44:35,544] [13] [common] - INFO - YAML file loaded successfully: params.yaml
[2025-06-09 11:44:35,549] [26] [common] - INFO - Created directory at: artifacts
[2025-06-09 11:44:35,552] [26] [common] - INFO - Created directory at: artifacts\training
[2025-06-09 11:44:35,555] [26] [common] - INFO - Created directory at: artifacts\logs
[2025-06-09 11:44:42,176] [108] [callback_connector] - INFO - Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
[2025-06-09 11:44:42,210] [156] [setup] - INFO - GPU available: True (cuda), used: True
[2025-06-09 11:44:42,212] [159] [setup] - INFO - TPU available: False, using: 0 TPU cores
[2025-06-09 11:44:42,213] [169] [setup] - INFO - HPU available: False, using: 0 HPUs
[2025-06-09 11:44:42,216] [166] [cuda] - INFO - You are using a CUDA 

d:\python-projects\chest-cancer-classification\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


d:\python-projects\chest-cancer-classification\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 4: 100%|██████████| 18/18 [00:11<00:00,  1.50it/s, v_num=cd8c, val_loss=0.901, val_accuracy=0.412, train_loss=0.877]
